In [2]:
import os

from configparser import ConfigParser

from snowflake.connector import connect

import pandas as pd

def bool_to_app_instalada(input: bool) -> str:
    return 'PUSH' if input else 'EMAIL'

config = ConfigParser()
config.read('config.ini')

conn = connect(
    user = config.get('SNOWFLAKE', 'USER'),
    password = config.get('SNOWFLAKE', 'PASSWORD'),
    account = config.get('SNOWFLAKE', 'ACCOUNT'),
    database = config.get('SNOWFLAKE', 'DATABASE'),
    warehouse = config.get('SNOWFLAKE', 'WAREHOUSE'),
    schema = config.get('SNOWFLAKE', 'SCHEMA'),
    role = config.get('SNOWFLAKE', 'ROLE'),
)

cur = conn.cursor()
cur.execute("""
    SELECT
        *
    FROM
        WOW_REWARDS.SEGMENTACION_DOMINOS.SEGMENTACION_JULIO
    WHERE
        SEMANA = 46
    AND
    (
        PHONENUMBER IS NOT null
        AND
        APP_INSTALADA
    )
    AND
        NAME IS NOT null
    ORDER BY
        PHONENUMBER
    -- PUSH + EMAIL
    ;
""")

df = cur.fetch_pandas_all()

df['TIPO'] = 'PUSH+EMAIL'

print(df.head())

df_list:list[tuple[tuple[str, str, str], pd.DataFrame]] = [tup for tup in df.groupby(['GRUPO', 'SEGMENTO', 'TIPO'])]

for item in df_list:
    name =  'Segmentacion_post_dominosmania_noviembre' + '_' + item[0][0] + '_' + item[0][1] + '_' + item[0][2] + '.csv'

    dir = './' + item[0][0]
    if not os.path.exists(dir):
        os.mkdir(dir)

    dir = './' + item[0][0] + '/' + item[0][2]
    if not os.path.exists(dir):
        os.mkdir(dir)

    item[1].to_csv(dir + '/' + name, index=False)

cur = conn.cursor()
cur.execute("""
    SELECT
        *
    FROM
        WOW_REWARDS.SEGMENTACION_DOMINOS.SEGMENTACION_JULIO
    WHERE
        SEMANA = 44
    AND
    (
        PHONENUMBER IS null
        OR
        NOT APP_INSTALADA
    )
    ;
""")

df = cur.fetch_pandas_all()

df['TIPO'] = 'SOLO_EMAIL'

print(df.head())

df_list:list[tuple[tuple[str, str, str], pd.DataFrame]] = [tup for tup in df.groupby(['GRUPO', 'SEGMENTO', 'TIPO'])]

for item in df_list:
    name =  'Segmentacion_post_dominosmania_noviembre' + '_' + item[0][0] + '_' + item[0][1] + '_' + item[0][2] + '.csv'

    dir = './' + item[0][0]
    if not os.path.exists(dir):
        os.mkdir(dir)

    dir = './' + item[0][0] + '/' + item[0][2]
    if not os.path.exists(dir):
        os.mkdir(dir)

    item[1].to_csv(dir + '/' + name, index=False)

    cur = conn.cursor()

   ANIO  SEMANA                              PROMO                      EMAIL  \
0  2023      46  PROMO_POST_DOMINOSMANIA_NOVIEMBRE   galleta.sesual@gmail.com   
1  2023      46  PROMO_POST_DOMINOSMANIA_NOVIEMBRE  diegomanjarrez3@gmail.com   
2  2023      46  PROMO_POST_DOMINOSMANIA_NOVIEMBRE      lpadilla001@gmail.com   
3  2023      46  PROMO_POST_DOMINOSMANIA_NOVIEMBRE      gatobeto.25@gmail.com   
4  2023      46  PROMO_POST_DOMINOSMANIA_NOVIEMBRE  chavezvictoriop@gmail.com   

  PHONENUMBER         NAME      GRUPO           SEGMENTO  APP_INSTALADA  \
0  0000000000       Evelyn  PROMOCION          INACTIVOS           True   
1  0000000000        Diego  PROMOCION  MEDIUM_HIGH_VALUE           True   
2  0000000000        Lidia  PROMOCION          LOW_VALUE           True   
3  0000000000      Alberto  PROMOCION   MEDIUM_LOW_VALUE           True   
4  0000000000  Paulo Cesar  PROMOCION          INACTIVOS           True   

  TIEMPO_DE_ENTREGA        TIPO  
0               <30  PUSH+EM

In [7]:
usuarios_por_ciudad = pd.concat([
    pd.read_csv('./usuarios_con_ciudad_activos.csv'),
    pd.read_csv('./usuarios_con_ciudad_inactivos.csv')
])

2148082
2148082


In [18]:
push_email_promocion = pd.concat([
    pd.read_csv('PROMOCION/PUSH+EMAIL/Segmentacion_post_dominosmania_noviembre_PROMOCION_LOW_VALUE_PUSH+EMAIL.csv'),
    pd.read_csv('PROMOCION/PUSH+EMAIL/Segmentacion_post_dominosmania_noviembre_PROMOCION_MEDIUM_LOW_VALUE_PUSH+EMAIL.csv'),
    pd.read_csv('PROMOCION/PUSH+EMAIL/Segmentacion_post_dominosmania_noviembre_PROMOCION_MEDIUM_HIGH_VALUE_PUSH+EMAIL.csv'),
    pd.read_csv('PROMOCION/PUSH+EMAIL/Segmentacion_post_dominosmania_noviembre_PROMOCION_INACTIVOS_PUSH+EMAIL.csv')
])

push_email_promocion_con_ciudad = pd.merge(push_email_promocion, usuarios_por_ciudad, on = 'EMAIL')
push_email_promocion_sin_ciudad = push_email_promocion[~push_email_promocion['EMAIL'].isin(usuarios_por_ciudad['EMAIL'])]

push_email_promocion_con_ciudad.to_csv('PROMOCION/PUSH+EMAIL/Segmentacion_post_dominosmania_noviembre_CON_CIUDAD.csv')
push_email_promocion_sin_ciudad.to_csv('PROMOCION/PUSH+EMAIL/Segmentacion_post_dominosmania_noviembre_SIN_CIUDAD.csv')

In [19]:
solo_email_promocion = pd.concat([
    pd.read_csv('PROMOCION/SOLO_EMAIL/Segmentacion_post_dominosmania_noviembre_PROMOCION_LOW_VALUE_SOLO_EMAIL.csv'),
    pd.read_csv('PROMOCION/SOLO_EMAIL/Segmentacion_post_dominosmania_noviembre_PROMOCION_MEDIUM_LOW_VALUE_SOLO_EMAIL.csv'),
    pd.read_csv('PROMOCION/SOLO_EMAIL/Segmentacion_post_dominosmania_noviembre_PROMOCION_MEDIUM_HIGH_VALUE_SOLO_EMAIL.csv'),
    pd.read_csv('PROMOCION/SOLO_EMAIL/Segmentacion_post_dominosmania_noviembre_PROMOCION_INACTIVOS_SOLO_EMAIL.csv')
])

solo_email_promocion_con_ciudad = pd.merge(solo_email_promocion, usuarios_por_ciudad, on = 'EMAIL')
solo_email_promocion_sin_ciudad = solo_email_promocion[~solo_email_promocion['EMAIL'].isin(usuarios_por_ciudad['EMAIL'])]

solo_email_promocion_con_ciudad.to_csv('PROMOCION/SOLO_EMAIL/Segmentacion_post_dominosmania_noviembre_CON_CIUDAD.csv')
solo_email_promocion_sin_ciudad.to_csv('PROMOCION/SOLO_EMAIL/Segmentacion_post_dominosmania_noviembre_SIN_CIUDAD.csv')

C:\Users\ITSC360 - 11\AppData\Local\Temp\ipykernel_21592\598638192.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv('PROMOCION/SOLO_EMAIL/Segmentacion_post_dominosmania_noviembre_PROMOCION_LOW_VALUE_SOLO_EMAIL.csv'),


In [21]:
dfs:dict[str, pd.DataFrame] = {}

dfs['LOW_VALUE'] = pd.read_csv('PROMOCION/PUSH+EMAIL/Segmentacion_post_dominosmania_noviembre_PROMOCION_LOW_VALUE_PUSH+EMAIL.csv')
dfs['MEDIUM_LOW_VALUE'] = pd.read_csv('PROMOCION/PUSH+EMAIL/Segmentacion_post_dominosmania_noviembre_PROMOCION_MEDIUM_LOW_VALUE_PUSH+EMAIL.csv')
dfs['MEDIUM_HIGH_VALUE'] = pd.read_csv('PROMOCION/PUSH+EMAIL/Segmentacion_post_dominosmania_noviembre_PROMOCION_MEDIUM_HIGH_VALUE_PUSH+EMAIL.csv')
dfs['INACTIVOS'] = pd.read_csv('PROMOCION/PUSH+EMAIL/Segmentacion_post_dominosmania_noviembre_PROMOCION_INACTIVOS_PUSH+EMAIL.csv')

for name in dfs:
    pd.merge(dfs[name], usuarios_por_ciudad, on = 'EMAIL', how = 'left').to_csv(f'PROMOCION/PUSH+EMAIL/Segmentacion_post_dominosmania_noviembre_PUSH_{name}.csv')

In [22]:
pd.concat([
    pd.read_csv('PROMOCION/PUSH+EMAIL/Segmentacion_post_dominosmania_noviembre_CON_CIUDAD.csv'),
    pd.read_csv('PROMOCION/SOLO_EMAIL/Segmentacion_post_dominosmania_noviembre_CON_CIUDAD.csv')
]).to_csv('./Segmentacion_post_dominosmania_noviembre_CON_CIUDAD.csv')

pd.concat([
    pd.read_csv('PROMOCION/PUSH+EMAIL/Segmentacion_post_dominosmania_noviembre_SIN_CIUDAD.csv'),
    pd.read_csv('PROMOCION/SOLO_EMAIL/Segmentacion_post_dominosmania_noviembre_SIN_CIUDAD.csv')
]).to_csv('./Segmentacion_post_dominosmania_noviembre_SIN_CIUDAD.csv')

C:\Users\ITSC360 - 11\AppData\Local\Temp\ipykernel_21592\396975891.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv('PROMOCION/SOLO_EMAIL/Segmentacion_post_dominosmania_noviembre_CON_CIUDAD.csv')
